In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow
!pip install tensorflow-text
!pip install simpletransformers
!pip install spacy
!pip install contractions
!pip install emoji
!pip install emot
!pip install demoji
!pip install nlpaug

In [ ]:
import pandas as pd
import spacy
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
import contractions
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import re
from nltk.tokenize import TweetTokenizer
import regex
import nltk
from nltk.stem import SnowballStemmer
import demoji
import emoji
from sklearn.metrics import f1_score
demoji.download_codes()
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report,accuracy_score
from sklearn.svm import LinearSVC

<ipython-input-4-782662059e0c>:26: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('whitespace')
nltk.download('wordnet')
nltk.download('omw-1.4')
stop_words = set(stopwords.words("english"))

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Training.tsv',sep='\t')

In [ ]:
data.head()

In [ ]:
data['label'].value_counts()

 # Preprocessing

In [ ]:
#emot_object = emot.core.emot()
ps =PorterStemmer()
lemmatiser = WordNetLemmatizer()
english_stopwords = stopwords.words('english')
exclude = set(string.punctuation)
def preprocess(text):
  #text=demoji.findall(df['Text'])
  text = contractions.fix(text.lower(), slang=True)
  text= re.sub(r'\d+', '', text)
  text=re.sub(r'$', '', text)
  text= re.sub(r'’','', text )
  text=re.sub('<.*?>','',text)
  text=re.sub(r'http\S+', '', text)
  #text=emoji.demojize(text, delimiters=(" ", " "))
  text = ''.join(ch for ch in text if ch not in exclude)
  tokens = word_tokenize(text)
  #print("Tokens:", tokens)
  text = [t for t in tokens if t not in english_stopwords]
  text = " ".join(text)
  return text

In [ ]:
import emoji
#import demoji
#demoji.download_codes()
def emo(text):
  temp=emoji.demojize(text,delimiters=(" "," "))
  temp=temp.replace("_","  ")
  return temp

In [ ]:
data['emo']=data["text"].apply(lambda x:emo(x))
data["clean_text"]=data['emo'].apply(lambda X: preprocess(X))
#added_data["clean_text"]

In [ ]:
data.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data['clean_text'], data['label'], test_size=0.33, random_state=42)

# Feature Extraction: TF-IDF (char_wb)

In [ ]:
Tfidf_vec = TfidfVectorizer(analyzer='char_wb', ngram_range=(1, 5), max_df=1.0, min_df=1, max_features=5000)
count_train = Tfidf_vec.fit(X_train)
train_features1 = Tfidf_vec.transform(X_train)
test_features1 = Tfidf_vec.transform(X_test)

**Feature Extraction- (word) TFIDF**

In [ ]:
# Tfidf_vec1 = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), max_df=1.0, min_df=1, max_features=5000)
# count_train1 = Tfidf_vec1.fit(X_train)
# train_features2= Tfidf_vec1.transform(X_train)
# test_features2 = Tfidf_vec1.transform(X_test)

 # Model Building with SVM - LinearSVC

In [ ]:
clf1 =LinearSVC(C=1.0, class_weight="balanced", max_iter=10000, random_state=123)
clf1.fit(train_features1, y_train)
y_pred1=clf1.predict(test_features1)
accuracy = accuracy_score(y_test, y_pred1)

print("Test Accuracy:", round(accuracy*100, 4))

print("\n", classification_report(y_test, y_pred1))

In [ ]:
clf1 =LinearSVC(C=1.0, class_weight="balanced", max_iter=10000, random_state=123)
clf1.fit(train_features2, y_train)
y_pred2=clf1.predict(test_features2)
accuracy = accuracy_score(y_test, y_pred2)

print("Test Accuracy:", round(accuracy*100, 4))

print("\n", classification_report(y_test, y_pred2))